この問題をbashで解決する方法を、詳細な図解とともに説明します。

In [ ]:
#!/bin/bash

# file.txtの内容を転置する

# 方法1: awkを使用したシンプルな解法
# Analyze Complexity
# Runtime 72 ms
# Beats 29.70%
# Memory 7.82 MB
# Beats 13.13%
awk '
{
    # 各フィールドを配列に格納
    for (i = 1; i <= NF; i++) {
        a[NR, i] = $i
    }
}
NF > p { p = NF }  # 最大列数を記録
END {
    # 転置して出力
    for (j = 1; j <= p; j++) {
        str = a[1, j]
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]
        }
        print str
    }
}
' file.txt

## 問題の理解

この問題は**行列の転置**と同じ操作です。元のファイルの行と列を入れ替えます。

### 入力例の視覚化

```
元のファイル (2列 × 3行):
┌─────────────┐
│ name   age  │  ← 行1
│ alice  21   │  ← 行2
│ ryan   30   │  ← 行3
└─────────────┘
   ↑      ↑
  列1    列2
```

### 出力の視覚化

```
転置後 (3列 × 2行):
┌────────────────────┐
│ name  alice  ryan  │  ← 元の列1が行1に
│ age   21     30    │  ← 元の列2が行2に
└────────────────────┘
```

## 解法の詳細説明

### **ステップ1: データの読み込みと配列への格納**

```
awk '{
    for (i = 1; i <= NF; i++) {
        a[NR, i] = $i
    }
}'
```

**図解:**

```
読み込み処理:

NR=1: name age
      ↓    ↓
   a[1,1] a[1,2]
    name   age

NR=2: alice 21
      ↓     ↓
   a[2,1] a[2,2]
   alice   21

NR=3: ryan 30
      ↓    ↓
   a[3,1] a[3,2]
   ryan    30

結果の2次元配列:
      列1    列2
    ┌──────┬─────┐
行1 │ name │ age │
    ├──────┼─────┤
行2 │alice │ 21  │
    ├──────┼─────┤
行3 │ ryan │ 30  │
    └──────┴─────┘
```

**変数の説明:**
- `NR`: 現在の行番号 (Number of Records)
- `NF`: 現在の行のフィールド数 (Number of Fields)
- `$i`: i番目のフィールド
- `a[NR, i]`: 2次元配列 (行, 列)

### **ステップ2: 最大列数の記録**

```
NF > p { p = NF }
```

各行のフィールド数をチェックし、最大値を`p`に保存します。

### **ステップ3: 転置して出力**

```
END {
    for (j = 1; j <= p; j++) {
        str = a[1, j]
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]
        }
        print str
    }
}
```

**図解:**

```
転置処理:

j=1のとき (元の列1):
┌─────┐
│name │ a[1,1]  →  str = "name"
├─────┤
│alice│ a[2,1]  →  str = "name alice"
├─────┤
│ryan │ a[3,1]  →  str = "name alice ryan"
└─────┘
出力: "name alice ryan"

j=2のとき (元の列2):
┌─────┐
│age  │ a[1,2]  →  str = "age"
├─────┤
│21   │ a[2,2]  →  str = "age 21"
├─────┤
│30   │ a[3,2]  →  str = "age 21 30"
└─────┘
出力: "age 21 30"
```

## 別解: より簡潔なワンライナー

```bash
awk '{for(i=1;i<=NF;i++)a[i]=a[i]" "$i}END{for(i=1;i<=NF;i++)print substr(a[i],2)}' file.txt
```

この方法では、各列のデータを文字列として連結していきます。

## 実行方法

```bash
# スクリプトを実行権限付きで保存
chmod +x transpose.sh

# 実行
./transpose.sh

# または直接awkコマンドを実行
awk '{for(i=1;i<=NF;i++)a[NR,i]=$i}NF>p{p=NF}END{for(j=1;j<=p;j++){str=a[1,j];for(i=2;i<=NR;i++)str=str" "a[i,j];print str}}' file.txt
```

## まとめ

この解法のポイント:
1. **2次元配列**を使ってデータを格納
2. **外側ループ**で列を、**内側ループ**で行を処理
3. 行と列のインデックスを入れ替えることで転置を実現

この手法は任意のサイズの行列に対応できます!

現在の解法は機能的には正しいですが、パフォーマンスとメモリ使用量に改善の余地があります。より効率的な解法を提案します。

In [ ]:
#!/bin/bash

# 最適化版: 文字列連結を使用してメモリ効率を改善
# Analyze Complexity
# Runtime 59 ms
# Beats 93.40%
# Memory 3.99 MB
# Beats 91.05%

awk '
{
    for (i = 1; i <= NF; i++) {
        if (NR == 1) {
            # 最初の行: 配列を初期化
            a[i] = $i
        } else {
            # 2行目以降: スペース区切りで連結
            a[i] = a[i] " " $i
        }
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]
    }
}
' file.txt

## パフォーマンス比較と改善点

### **元の解法の問題点**

```
問題1: 2次元配列の使用
┌─────────────────────┐
│ a[1,1] a[1,2]       │  メモリ使用量: O(行数 × 列数)
│ a[2,1] a[2,2]       │  各セルを個別に保存
│ a[3,1] a[3,2]       │  アクセスオーバーヘッドが大きい
└─────────────────────┘

問題2: 二重ループでの文字列連結
END {
    for (j = 1; j <= p; j++) {
        str = a[1, j]              ← 毎回新しい文字列を作成
        for (i = 2; i <= NR; i++) {
            str = str " " a[i, j]  ← 文字列を何度も再構築
        }
        print str
    }
}
```

### **改善版の利点**

```
改善1: 1次元配列 + 文字列連結
┌──────────────────────┐
│ a[1] = "name alice ryan" │  メモリ: O(列数)のみ
│ a[2] = "age 21 30"       │  各列を1つの文字列として保存
└──────────────────────────┘

改善2: 処理中に連結
{
    for (i = 1; i <= NF; i++) {
        a[i] = a[i] " " $i  ← 読み込みながら連結
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]           ← 単純な出力のみ
    }
}
```

## さらなる最適化オプション

### **オプション1: rs コマンド (最速)**

```bash
# rsコマンドを使用 (利用可能な場合)
rs -T < file.txt
```

**特徴:**
- C言語で実装された専用ツール
- 最も高速
- ただし、一部のシステムでは利用不可

### **オプション2: 改良版awkスクリプト**

In [ ]:
#!/bin/bash

# 方法1: 最適化版 (推奨) - メモリ効率が良い
# Analyze Complexity
# Runtime 63 ms
# Beats 77.60%
# Memory 3.99 MB
# Beats 91.05%

awk '{for(i=1;i<=NF;i++)a[i]=a[i](NR>1?" ":"")$i}END{for(i=1;i<=NF;i++)print a[i]}' file.txt

# 方法2: 可読性重視版
# Analyze Complexity
# Runtime 67 ms
# Beats 53.43%
# Memory 3.98 MB
# Beats 91.05%

awk '
{
    for (i = 1; i <= NF; i++) {
        a[i] = a[i] (NR > 1 ? " " : "") $i
    }
}
END {
    for (i = 1; i <= NF; i++) {
        print a[i]
    }
}
' file.txt

# 方法3: pasteコマンド利用 (Bash純正アプローチ)
# Analyze Complexity
# Runtime 269 ms
# Beats 5.64%
# Memory 3.59 MB
# Beats 100.00%

cols=$(head -1 file.txt | wc -w)
for i in $(seq 1 $cols); do
    cut -d' ' -f$i file.txt | paste -sd' '
done

## パフォーマンス分析

### **時間計算量の比較**

```
元の解法:
- 読み込み: O(行数 × 列数)
- 出力: O(行数 × 列数)
- 合計: O(2 × 行数 × 列数)

改善版:
- 読み込み+連結: O(行数 × 列数)
- 出力: O(列数)
- 合計: O(行数 × 列数 + 列数)
```

### **メモリ使用量の比較**

```
元の解法:
┌──────────────────────────────┐
│ 2次元配列: 行数 × 列数 個の要素  │
│ 一時文字列: 列数 個             │
│ 合計: O(行数 × 列数)           │
└──────────────────────────────┘

改善版:
┌──────────────────────────────┐
│ 1次元配列: 列数 個の文字列       │
│ 各文字列長: 行数 × 平均単語長    │
│ 合計: O(列数 × 行数) だが       │
│       実装が効率的             │
└──────────────────────────────┘
```

### **実行例の詳細図解**

```
入力: file.txt
┌─────────────┐
│ name   age  │
│ alice  21   │
│ ryan   30   │
└─────────────┘

処理フロー:

NR=1: name age
      ↓    ↓
   a[1]="name"
   a[2]="age"

NR=2: alice 21
      ↓     ↓
   a[1]="name alice"    ← スペース追加して連結
   a[2]="age 21"

NR=3: ryan 30
      ↓    ↓
   a[1]="name alice ryan"
   a[2]="age 21 30"

END処理:
   print a[1]  → "name alice ryan"
   print a[2]  → "age 21 30"
```

## ベンチマーク予想

改善版の期待値:
- **Runtime**: 40-50ms (約40-50%改善)
- **Memory**: 4-5MB (約35-40%改善)

### **主な改善要因**

1. **配列アクセスの削減**: 2次元→1次元
2. **ループネストの削減**: 二重ループ→単一ループ
3. **文字列操作の最適化**: 逐次連結→直接連結
4. **条件分岐の最適化**: 三項演算子の使用

この最適化により、上位50-70%のパフォーマンスが期待できます!